<a href="https://colab.research.google.com/github/IbrahimIsman/WebScrapingPrius/blob/main/Web_scraping_prius_cars_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd


In [ ]:
website = "https://www.sbtjapan.com/used-cars/toyota/prius/?steering=right-hand-drive&drive=0&year_f=2015&year_t=2023&cc_f=0&cc_t=0&mile_f=11&mile_t=70&trans=0&savel=0&saveu=0&fuel=0&color=0&bodyLength=0&loadClass=0&engineType=0&location=&port=0&search_box=1&locationIds=0&d_country=2&d_port=65&ship_type=0&FreightChk=yes&currency=2&inspection=yes&insurance=1&sort=5&psize=100&p_num=2#listbox"

In [ ]:
response =  requests.get(website)

In [ ]:
response.status_code

In [ ]:
 Soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
results = Soup.find_all('div', {'class': 'car_info_area popbox'})

In [ ]:
len(results)

In [ ]:
#@title
results[0]

In [ ]:
results[0].find('h2').get_text().strip()

In [ ]:
results[0].find('li').get_text().strip()

In [ ]:
results[0].find('p', {'style' : "font-weight: 700;"}).get_text().strip()

In [ ]:
name = []
mileage = []
price = []

for result in results:
  try:
    name.append(result.find('h2').get_text().strip())
  except:
    name.append('n/a')

  try:
    mileage.append(result.find('li').get_text().strip())
  except:
    mileage.append('n/a')

  try:
    price.append(result.find('p', {'style' : "font-weight: 700;"}).get_text().strip())
  except:
    price.append('n/a')


**Adding engine size**

In [ ]:
 Soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
results = Soup.find_all('div', {'class': 'car_info_area popbox'})

In [ ]:
#@title
results[0]

In [ ]:
results_span = []



In [ ]:
for r in results:
  results_span.append(r.find_all('span'))


In [ ]:
len(results_span)

In [ ]:
results_engine =[]
for  r in results_span:
  results_engine.append(r[2])

In [ ]:
len(results_engine)

**Converting to Pandas DF**




In [ ]:
prius_info = pd.DataFrame({'Name': name, 'Mileage': mileage, 'Price': price, 'Engine':results_engine})

**Data cleaning**

In [ ]:
prius_info.head()

In [ ]:
prius_info['Mileage'] = prius_info['Mileage'].apply(lambda x:x.strip('Mileage\n'))

In [ ]:
prius_info['Mileage'] = prius_info['Mileage'].apply(lambda x:x.strip('km'))

In [ ]:
prius_info['Price'] = prius_info['Price'].apply(lambda x:x.strip('USD'))

In [ ]:
prius_info

**Feature engineering**

In [ ]:
prius_info_split_name = pd.DataFrame(prius_info.Name.str.split(' ',1).tolist(),
                                 columns = ['Release date','Name'])

In [ ]:
Prius_droppedCol = prius_info.drop('Name', axis=1)

In [ ]:
concatenated = pd.concat([prius_info_split_name, Prius_droppedCol], axis="columns")

In [ ]:
cleaned_name = concatenated['Name'].str.encode('ascii', 'ignore').str.decode('ascii')

In [ ]:
concatenated = concatenated.drop('Name', axis=1)

In [ ]:
Final = pd.concat([cleaned_name, concatenated], axis="columns")

In [ ]:
Final.to_excel('search_page2.xlsx', index=False)


**Looping through all pages**

In [ ]:
websites = []

for i in range(2,14):
  website = 'https://www.sbtjapan.com/used-cars/toyota/prius/?steering=right-hand-drive&drive=0&year_f=2015&year_t=2023&cc_f=0&cc_t=0&mile_f=11&mile_t=70&trans=0&savel=0&saveu=0&fuel=0&color=0&bodyLength=0&loadClass=0&engineType=0&location=&port=0&search_box=1&locationIds=0&d_country=2&d_port=65&ship_type=0&FreightChk=yes&currency=2&inspection=yes&insurance=1&sort=5&psize=100&p_num=' + str(i)+ '#listbox'
  websites.append(website)



In [ ]:

name = []
mileage = []
price = []

for w in websites:
  response =  requests.get(w)
  soup = BeautifulSoup(response.content, 'html.parser')
  result = soup.find_all('li', {'class': 'car_listitem'})

  for r in result:
    try:
      name.append(r.find('h2').get_text().strip())
    except:
      name.append('n/a')

    try:
      mileage.append(r.find('li').get_text().strip())
    except:
      mileage.append('n/a')

    try:
      price.append(r.find('p', {'style' : "font-weight: 700;"}).get_text().strip())
    except:
      price.append('n/a')

In [ ]:
len(mileage)

**Scraping the engine size**



In [ ]:
engine =[]
results_span_all = []

for w in websites:
  response =  requests.get(w)
  soup = BeautifulSoup(response.content, 'html.parser')
  result = soup.find_all('div', {'class': 'car_info_area popbox'})

  for r in result:
    results_span_all.append(r.find_all('span'))

In [ ]:
for r in results_span_all:
      engine.append(r[2])

In [ ]:
engine[:10]

**Converted to Pandas Dataframe**

In [ ]:
prius_info_all = pd.DataFrame({'Name': name, 'Mileage': mileage, 'Price': price, 'Engine': engine})

In [ ]:
prius_info_all.info()

**Data cleaning through all pages**

In [ ]:
prius_info_all['Mileage'] = prius_info_all['Mileage'].apply(lambda x:x.strip('Mileage\n'))
prius_info_all['Mileage'] = prius_info_all['Mileage'].apply(lambda x:x.strip('km'))
prius_info_all['Price'] = prius_info_all['Price'].apply(lambda x:x.strip('USD'))
prius_info_all['Name'] = prius_info_all['Name'].str.encode('ascii', 'ignore').str.decode('ascii')

**Feature engineering**

In [ ]:
prius_info_all_split_name = pd.DataFrame(prius_info_all.Name.str.split(' ',1).tolist(),columns = ['Release date','Name'])
Prius_info_all_droppedCol = prius_info_all.drop('Name', axis=1)
Final_all = pd.concat([prius_info_all_split_name, Prius_info_all_droppedCol], axis="columns")

In [ ]:
Final_all.info()

**Converting to excel sheet**

In [ ]:
Final_all.to_excel('pages 2-13.xlsx', index=False)

**Appending search page 1 with pages 2 to 13**

In [ ]:
Combined_all = Final.append(
      Final_all, ignore_index=True)



In [ ]:
Combined_all.info()

In [ ]:
Combined_all

**Converting all to excel**

In [ ]:
Combined_all.to_excel('sbtjapan Prius info.xlsx', index=False)
